# =================== COMEÇO ==========================

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import unicodedata
import math
import pickle
from IPython.core.display  import HTML
from matplotlib import pyplot as plt
from sklearn                 import model_selection as ms
from sklearn                 import preprocessing as pp
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.ensemble      import RandomForestRegressor
from sklearn.metrics       import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
import xgboost as xgb
import lightgbm as lgbm
import catboost as catb

from boruta import BorutaPy
from scipy.stats import expon, randint

import warnings
warnings.filterwarnings( 'ignore' )
from data_preprocessing import preprocess_data, feature_engineering, main
from transformation import Transformation


# Helper Functions

In [10]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.rcParams['figure.figsize'] = [25, 15]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container {width:90% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set_theme(style="darkgrid", palette='hls')
    # sns.color_palette("crest", as_cmap=True)
    # sns.color_palette("magma", as_cmap=True)
    # sns.color_palette("flare", as_cmap=True)
    # Increase chart size for better readability
    #sns.set(rc={'figure.figsize':(11,6)})

jupyter_settings()

def mean_percentage_error( y, yhat ):
    return np.mean( ( y - yhat ) / y )

def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )

    return pd.DataFrame( { 'Model Name': [model_name],
                           'MAE': [mae],
                           'MAPE': [mape],
                           'RMSE': [rmse]} )

def cross_validation_regression(model, x_train, y_train, k, verbose=True):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    mae_list = []
    mape_list = []
    rmse_list = []
    i = 1    
    for train_cv, val_cv in kf.split(x_train, y_train):
        if verbose:
            print(f'Fold Number {i}/{k}') 
        else:
            pass

        x_train_fold = x_train.iloc[train_cv]
        y_train_fold = y_train.iloc[train_cv]

        x_val_fold = x_train.iloc[val_cv]
        y_val_fold = y_train.iloc[val_cv]

        # fit model
        model.fit(x_train_fold, y_train_fold)

        # predict
        yhat_model = model.predict(x_val_fold)

        #performance
        m_result = ml_error( model, np.expm1( y_val_fold ), np.expm1( yhat_model ) )
        
        # Calculate regression metrics for the fold
        mae_list.append( m_result[ 'MAE' ] )
        mape_list.append( m_result[ 'MAPE' ] )
        rmse_list.append( m_result[ 'RMSE' ] )
       
        i += 1

    results_dict = {'Model Name': type(model).__name__,
                       'MAE Mean': np.round(np.mean(mae_list), 4),
                       'MAPE Mean': np.round(np.mean(mape_list), 4),
                       'RMSE Mean': np.round(np.mean(rmse_list), 4)}
    return results_dict

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [11]:
# Load cars_test
cars_test = pd.read_csv('data/cars_test.csv', encoding='utf-16', delimiter='\t')

# Preprocess data
df1_test = preprocess_data(cars_test)

# Feature engineering
df_test_pre = feature_engineering(df1_test)

In [14]:
mean_ano_modelo = df_test_pre['ano_modelo'].mean()
mean_ano_modelo

2017.801662948692

In [6]:
df_test_pre.head(3)

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,...,anunciante,entrega_delivery,troca,dono_aceita_troca,veiculo_unico_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fabrica,revisoes_dentro_agenda
0,13518783164498355150900635905895481162,8,NISSAN,VERSA,1.6 16V FLEXSTART V-DRIVE MANUAL,2021,2021,20258.0,Manual,4,...,Pessoa Física,0,0,1,0,0,1,1,1,0
1,299896161723793383246777788797566040330,18,FIAT,STRADA,1.4 MPI WORKING CS 8V FLEX 2P MANUAL,2021,2021,53045.0,Manual,2,...,Loja,0,0,1,1,0,1,1,0,0
2,316180649972302128246133616457018378621,8,AUDI,Q5,2.0 TFSI GASOLINA BLACK S TRONIC,2018,2019,32486.0,Automática,4,...,Pessoa Física,0,0,1,0,0,1,0,0,0


In [115]:
df_test_pre.cambio.unique()

array(['Manual', 'Automática', 'CVT', 'Automatizada',
       'Automática Sequencial', 'Automatizada DCT', 'Semi-automática'],
      dtype=object)

In [22]:
df_test_pre.shape

(9862, 26)

In [12]:
# Load cars_train
cars_train_raw = pd.read_csv('data/cars_train.csv', encoding='utf-16', delimiter='\t')

# Preprocess data
df1 = preprocess_data(cars_train_raw)

# Feature engineering
df_train_pre = feature_engineering(df1)

In [24]:
df_train_pre.shape

(29584, 27)

In [117]:
df_train_pre.cambio.unique()

array(['CVT', 'Automática', 'Manual', 'Automatizada', 'Semi-automática',
       'Automatizada DCT', 'Automática Sequencial'], dtype=object)

In [12]:
df_train_pre.columns

Index(['id', 'num_fotos', 'marca', 'modelo', 'versao', 'ano_de_fabricacao',
       'ano_modelo', 'hodometro', 'cambio', 'num_portas', 'tipo', 'blindado',
       'cor', 'tipo_vendedor', 'cidade_vendedor', 'estado_vendedor',
       'anunciante', 'entrega_delivery', 'troca', 'dono_aceita_troca',
       'veiculo_unico_dono', 'revisoes_concessionaria', 'ipva_pago',
       'veiculo_licenciado', 'garantia_de_fabrica', 'revisoes_dentro_agenda',
       'preco'],
      dtype='object')

# Split Dataset cars_train into train and validation

In [13]:
X = df_train_pre.drop( ['id', 'preco'], axis=1 )
y = df_train_pre['preco']

x_train, x_validation, y_train, y_validation = ms.train_test_split( X, y, test_size=0.20, random_state=42 )

df4 = pd.concat( [x_train, y_train], axis=1 )

# Encoding

In [14]:
mms_num_portas = pp.MinMaxScaler()
le_cambio = pp.LabelEncoder()
le_cidade_vendedor = pp.LabelEncoder()
le_estado_vendedor = pp.LabelEncoder()
le_anunciante = pp.LabelEncoder()
le_ano_de_fabricacao = pp.LabelEncoder()
le_ano_modelo = pp.LabelEncoder()

# num_portas
df4['num_portas'] = mms_num_portas.fit_transform( df4[['num_portas']].values )
pickle.dump( mms_num_portas, open('/Users/Luan/repos/lighthouse/features/num_portas_scaler.pkl', 'wb' ) )

# 'cambio' Label Encoder
df4['cambio'] = le_cambio.fit_transform( df4['cambio'] )
pickle.dump( le_cambio, open( '/Users/Luan/repos/lighthouse/features/cambio_scaler.pkl', 'wb' ) )

# 'tipo' Ordinal Encoding
tipo_dict = {'Hatchback':1, 'Picape':2, 'Utilitário esportivo':3, 'Sedã':4, 'Cupê':5, 'Perua/SW':6, 'Minivan':7, 'Conversível':8}
df4['tipo'] = df4['tipo'].map( tipo_dict )

# 'cor' Ordinal Encoding
cor_dict = {'Branco': 1 , 'Preto': 2, 'Prata': 3, 'Cinza': 4, 'Verde': 5, 'Vermelho': 6, 'Dourado':7, 'Azul':8}
df4['cor'] = df4['cor'].map( cor_dict )

# 'cidade_vendedor' Label Encoder
df_train_pre['cidade_vendedor'] = le_cidade_vendedor.fit_transform( df_train_pre['cidade_vendedor'] )
df4['cidade_vendedor'] = le_cidade_vendedor.transform( df4['cidade_vendedor'] )
pickle.dump( le_cidade_vendedor, open( '/Users/Luan/repos/lighthouse/features/cidade_vendedor_scaler.pkl', 'wb' ) )

# 'estado_vendedor' Label Encoder
df4['estado_vendedor'] = le_estado_vendedor.fit_transform( df4['estado_vendedor'] )
pickle.dump( le_estado_vendedor, open( '/Users/Luan/repos/lighthouse/features/estado_vendedor_scaler.pkl', 'wb' ) )

# 'anunciante' Label Encoder
df4['anunciante'] = le_anunciante.fit_transform( df4['anunciante'] )
pickle.dump( le_anunciante, open( '/Users/Luan/repos/lighthouse/features/anunciante_scaler.pkl', 'wb' ) )

# 'tipo_vendedor' One-Hot Encoding
df4 = pd.get_dummies( df4, prefix=['tipo_vendedor'], columns=['tipo_vendedor'] )

# marca - Target Encoding
target_encode_marca = df4.groupby( 'marca' )['preco'].mean()
df4.loc[:, 'marca'] = df4['marca'].map( target_encode_marca )
pickle.dump( target_encode_marca, open( '/Users/Luan/repos/lighthouse/features/target_encode_marca_scaler.pkl', 'wb' ) )

# modelo - Target Encoding
target_encode_modelo = df4.groupby( 'modelo' )['preco'].mean()
df4.loc[:, 'modelo'] = df4['modelo'].map( target_encode_modelo )
pickle.dump( target_encode_modelo, open( '/Users/Luan/repos/lighthouse/features/target_encode_modelo_scaler.pkl', 'wb' ) )

# versao - Target Encoding
target_encode_versao = df4.groupby( 'versao' )['preco'].mean()
df4.loc[:, 'versao'] = df4['versao'].map( target_encode_versao )
pickle.dump( target_encode_versao, open( '/Users/Luan/repos/lighthouse/features/target_encode_versao_scaler.pkl', 'wb' ) )

# 'ano_de_fabricacao' Label Encoder
df_train_pre['ano_de_fabricacao'] = le_ano_de_fabricacao.fit_transform( df_train_pre['ano_de_fabricacao'] )
df4['ano_de_fabricacao'] = le_ano_de_fabricacao.transform( df4['ano_de_fabricacao'] )
pickle.dump( le_ano_de_fabricacao, open( '/Users/Luan/repos/lighthouse/features/ano_de_fabricacao_scaler.pkl', 'wb' ) )

# 'ano_modelo' Label Encoder
df_train_pre['ano_modelo'] = le_ano_modelo.fit_transform( df_train_pre['ano_modelo'] )
df4['ano_modelo'] = le_ano_modelo.transform( df4['ano_modelo'] )
pickle.dump( le_ano_modelo, open( '/Users/Luan/repos/lighthouse/features/ano_modelo_scaler.pkl', 'wb' ) )
mean_ano_modelo = df4['ano_modelo'].mean()
pickle.dump( mean_ano_modelo, open( '/Users/Luan/repos/lighthouse/features/mean_ano_modelo.pkl', 'wb' ) )

In [15]:
df4.ano_modelo.unique()

array([11, 10,  6,  8, 15, 14,  7, 13, 12,  5,  9,  4, 16,  3,  2,  1],
      dtype=int64)

# Response Variable Transformation

A variável resposta possui uma cauda longa à direita, por isso será aplicado transformação logarítmica

In [15]:
# aplicando log a base de treino 
df4['preco'] = np.log1p( df4['preco'] )

# aplicando log ao teste
y_validation = np.log1p( y_validation )

# Validation Preparation

In [16]:
# num_portas
x_validation['num_portas'] = mms_num_portas.transform( x_validation[['num_portas']].values )

# 'cambio' Label Encoder
x_validation['cambio'] = le_cambio.transform( x_validation['cambio'] )

# 'tipo' Ordinal Encoding
tipo_dict = {'Hatchback':1, 'Picape':2, 'Utilitário esportivo':3, 'Sedã':4, 'Cupê':5, 'Perua/SW':6, 'Minivan':7, 'Conversível':8}
x_validation['tipo'] = x_validation['tipo'].map( tipo_dict )

# 'cor' Ordinal Encoding
cor_dict = {'Branco': 1 , 'Preto': 2, 'Prata': 3, 'Cinza': 4, 'Verde': 5, 'Vermelho': 6, 'Dourado':7, 'Azul':8}
x_validation['cor'] = x_validation['cor'].map( cor_dict )

# 'cidade_vendedor' Label Encoder
x_validation['cidade_vendedor'] = le_cidade_vendedor.transform( x_validation['cidade_vendedor'] )


# 'estado_vendedor' Label Encoder
x_validation['estado_vendedor'] = le_estado_vendedor.transform( x_validation['estado_vendedor'] )

# 'anunciante' Label Encoder
x_validation['anunciante'] = le_anunciante.transform( x_validation['anunciante'] )

# 'tipo_vendedor' One-Hot Encoding
x_validation = pd.get_dummies( x_validation, prefix=['tipo_vendedor'], columns=['tipo_vendedor'] )

# marca - Target Encoding
x_validation.loc[:, 'marca'] = x_validation['marca'].map( target_encode_marca )

# modelo - Target Encoding
x_validation.loc[:, 'modelo'] = x_validation['modelo'].map( target_encode_modelo )

# versao - Target Encoding
x_validation.loc[:, 'versao'] = x_validation['versao'].map( target_encode_versao )

# 'ano_de_fabricacao' Label Encoder
x_validation['ano_de_fabricacao'] = le_ano_de_fabricacao.transform( x_validation['ano_de_fabricacao'] )

# 'ano_modelo' Label Encoder
x_validation['ano_modelo'] = le_ano_modelo.transform( x_validation['ano_modelo'] )


x_validation = x_validation.fillna( 0 )

# Feature Selection

In [ ]:
# Criando o modelo de regressão de floresta aleatória
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# data preparation
x_train_n = x_train.copy()
y_train_n = y_train.values

# Treinando o modelo nos dados de treinamento
rf_model.fit( x_train_n, y_train_n )

importances = rf_model.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_model.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature Ranking:")
df = pd.DataFrame()
for i, j in zip( x_train_n, rf_model.feature_importances_ ):
    aux = pd.DataFrame( {'feature': i, 'importance': j}, index=[0] )
    df = pd.concat([df, aux], axis=0 )
    
print( df.sort_values( 'importance', ascending=False ) )

# Machine Learning Modelling

In [17]:
x_train = df4.drop(['preco'], axis=1)
y_train = df4['preco']

x_test = x_validation.copy()
y_test = y_validation.copy()

## Linear Regression Model

In [38]:
# model
lr_model = LinearRegression().fit( x_train, y_train )

# prediction
yhat_lr = lr_model.predict( x_test )

# performance
lr_result = ml_error( 'Linear Regression', np.expm1( y_test ), np.expm1( yhat_lr ) )
lr_result

,Model Name,MAE,MAPE,RMSE
0,Linear Regression,33341.668666,0.252228,67123.126572


## Random Forest Regressor

In [70]:
# model
rf_model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42 ).fit( x_train, y_train )

# prediction
yhat_rf = rf_model.predict( x_test )

# performance
rf_result = ml_error( 'Random Forest Regressor', np.expm1( y_test ), np.expm1( yhat_rf ) )
rf_result

,Model Name,MAE,MAPE,RMSE
0,Random Forest Regressor,27350.255069,0.219277,45640.499592


## XGBoost Regressor

In [71]:
# model
xgb_model = xgb.XGBRegressor().fit( x_train, y_train )

# prediction
yhat_xgb = xgb_model.predict( x_test )

# performance
xgb_result = ml_error( 'XGBoost Regressor', np.expm1( y_test ), np.expm1( yhat_xgb ) )
xgb_result

,Model Name,MAE,MAPE,RMSE
0,XGBoost Regressor,27180.376719,0.217619,45116.739434


## LGBM Regressor

In [72]:
# model
lgbm_model = lgbm.LGBMRegressor().fit( x_train, y_train )

# prediction
yhat_lgbm = lgbm_model.predict( x_test )

# performance
lgbm_result = ml_error( 'LGBM Regressor', np.expm1( y_test ), np.expm1( yhat_lgbm ) )
lgbm_result

,Model Name,MAE,MAPE,RMSE
0,LGBM Regressor,26928.02552,0.215214,44566.094111


## CatBoost Regressor

In [123]:
# model
catb_model = catb.CatBoostRegressor(verbose=False).fit( x_train, y_train )

# prediction
yhat_catb = catb_model.predict( x_test )

# performance
catb_result = ml_error( 'CatBoost Regressor', np.expm1( y_test ), np.expm1( yhat_catb ) )
catb_result

,Model Name,MAE,MAPE,RMSE
0,CatBoost Regressor,26366.406052,0.210202,43392.122279


## Cross Validation

In [74]:
models_list = [lr_model, rf_model, xgb_model, lgbm_model, catb_model]
results_list = []

for model in models_list:
    results_model = cross_validation_regression( model, x_train, y_train, 5, verbose=False )
    
    results_list.append(results_model)
    
results_cv = pd.DataFrame(results_list)
    
results_cv

,Model Name,MAE Mean,MAPE Mean,RMSE Mean
0,LinearRegression,33873.7485,0.2378,268643.7100
1,RandomForestRegressor,24600.2803,0.1945,40052.9735
2,XGBRegressor,24622.1625,0.1946,40288.7580
3,LGBMRegressor,24191.1182,0.1919,39286.5704
4,CatBoostRegressor,24012.8915,0.1903,39136.3274


# Hyperparameter Fine Tuning

In [16]:
## Crie o modelo de regressor CatBoost
#catboost_model = catb.CatBoostRegressor(iterations=100, random_state=42, verbose=0)
#
## Defina a grade de hiperparâmetros para a busca aleatória
#param_dist = {
#    'learning_rate': expon(loc=0.01, scale=0.2),
#    'depth': randint(3, 16),
#    'l2_leaf_reg': expon(loc=1, scale=10),
#    'bagging_temperature': expon(loc=0, scale=1),
#    'random_strength': expon(loc=0, scale=1),
#    'one_hot_max_size': randint(2, 10),
#}
#
## Realize a busca aleatória com o RandomizedSearchCV
#random_search = ms.RandomizedSearchCV(catboost_model, param_distributions=param_dist, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42, verbose=False)
#random_search.fit(x_train, y_train)
#
## Exiba os melhores hiperparâmetros encontrados e o desempenho do modelo
#print("Melhores hiperparâmetros:")
#print(random_search.best_params_)
#print("Melhor MSE:", -random_search.best_score_)

In [36]:
best_param = {'bagging_temperature': 1.9713449718798333, 'depth': 9, 'l2_leaf_reg': 2.799114871179353, 'learning_rate': 0.16275231006818694, 'one_hot_max_size': 7, 'random_strength': 0.2511673382758245}

In [18]:
# model
catb_model_tunned = catb.CatBoostRegressor( bagging_temperature= 1.9713449718798333, 
                                            depth= 9, 
                                            l2_leaf_reg= 2.799114871179353, 
                                            learning_rate= 0.16275231006818694, 
                                            one_hot_max_size= 7, 
                                            random_strength= 0.2511673382758245, verbose=False).fit( x_train, y_train )

# prediction
yhat_catb_tunned = catb_model_tunned.predict( x_test )

# performance
catb_result_tunned = ml_error( 'CatBoost Regressor', np.expm1( y_test ), np.expm1( yhat_catb_tunned ) )
catb_result_tunned

,Model Name,MAE,MAPE,RMSE
0,CatBoost Regressor,26607.963801,0.212053,43115.383203


In [19]:
# Save trained model
pickle.dump( catb_model_tunned, open('/Users/Luan/repos/lighthouse/models/catb_model_tunned.pkl', 'wb' ))

In [77]:
cross_validation_regression(catb_model_tunned, x_train, y_train, 5, verbose=False)

{'Model Name': 'CatBoostRegressor',
 'MAE Mean': 24662.2179,
 'MAPE Mean': 0.195,
 'RMSE Mean': 40151.6354}

In [ ]:
import pickle
import numpy as np
import pandas as pd

class Transformation( object ):
    def __init__( self ):
        self.home_path = '/Users/Luan/repos/lighthouse/'
        self.mms_num_portas = pickle.load( open( self.home_path + 'features/num_portas_scaler.pkl', 'rb'))
        self.le_cambio = pickle.load( open( self.home_path + 'features/cambio_scaler.pkl', 'rb'))
        self.le_tipo = pickle.load( open( self.home_path + 'features/tipo_scaler.pkl', 'rb'))
        self.le_cor = pickle.load( open( self.home_path + 'features/cor_scaler.pkl', 'rb'))
        self.le_cidade_vendedor = pickle.load( open( self.home_path + 'features/cidade_vendedor_scaler.pkl', 'rb'))
        self.le_estado_vendedor = pickle.load( open( self.home_path + 'features/estado_vendedor_scaler.pkl', 'rb'))
        self.le_anunciante = pickle.load( open( self.home_path + 'features/anunciante_scaler.pkl', 'rb'))
        self.le_ano_de_fabricacao = pickle.load( open( self.home_path + 'features/ano_de_fabricacao_scaler.pkl', 'rb'))
        self.le_ano_modelo = pickle.load( open( self.home_path + 'features/ano_modelo_scaler.pkl', 'rb'))
        self.target_encode_marca = pickle.load( open( self.home_path + 'features/target_encode_marca_scaler.pkl','rb'))
        self.target_encode_modelo = pickle.load( open( self.home_path + 'features/target_encode_modelo_scaler.pkl', 'rb'))
        self.target_encode_versao = pickle.load( open( self.home_path + 'features/target_encode_versao_scaler.pkl', 'rb'))
        
    def data_transform( self, df_pre ):
        # num_portas
        df_pre['num_portas'] = self.mms_num_portas.transform( df_pre[['num_portas']].values )

        # 'cambio' Label Encoder
        df_pre['cambio'] = self.le_cambio.transform( df_pre['cambio'] )

        # 'tipo' Label Encoder
        df_pre['tipo'] = self.le_tipo.transform( df_pre['tipo'] )

        # 'cor' Label Encoder
        df_pre['cor'] = self.le_cor.transform( df_pre['cor'] )

        # 'cidade_vendedor' Label Encoder
        df_pre['cidade_vendedor'] = self.le_cidade_vendedor.transform( df_pre['cidade_vendedor'] )


        # 'estado_vendedor' Label Encoder
        df_pre['estado_vendedor'] = self.le_estado_vendedor.transform( df_pre['estado_vendedor'] )

        # 'anunciante' Label Encoder
        df_pre['anunciante'] = self.le_anunciante.transform( df_pre['anunciante'] )

        # 'tipo_vendedor' One-Hot Encoding
        df_pre = pd.get_dummies( df_pre, prefix=['tipo_vendedor'], columns=['tipo_vendedor'] )

        # marca - Target Encoding
        df_pre.loc[:, 'marca'] = df_pre['marca'].map( self.target_encode_marca )

        # modelo - Target Encoding
        df_pre.loc[:, 'modelo'] = df_pre['modelo'].map( self.target_encode_modelo )

        # versao - Target Encoding
        df_pre.loc[:, 'versao'] = df_pre['versao'].map( self.target_encode_versao )

        # 'ano_de_fabricacao' Label Encoder
        df_pre['ano_de_fabricacao'] = self.le_ano_de_fabricacao.transform( df_pre['ano_de_fabricacao'] )

        # 'ano_modelo' Label Encoder
        df_pre['ano_modelo'] = self.le_ano_modelo.transform( df_pre['ano_modelo'] )


        df_pre = df_pre.fillna( 0 )
        
        return df_pre

In [97]:
df_test_pre.tipo.unique()

array(['Hatchback', 'Picape', 'Utilitário esportivo', 'Sedã', 'Cupê',
       'Perua/SW', 'Minivan', 'Conversível'], dtype=object)

In [20]:
transformer = Transformation()
df_test_transformed = transformer.data_transform(df_test_pre)


In [21]:
test = df_test_transformed.drop('id', axis=1)

# Predict

In [22]:
prediction = pd.DataFrame(catb_model_tunned.predict(test))

In [24]:
cars_test['preco'] = prediction
cars_test.head()

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco
0,13518783164498355150900635905895481162,8.0,NISSAN,VERSA,1.6 16V FLEXSTART V-DRIVE MANUAL,2021,2021.0,20258.0,Manual,4,Hatchback,N,Branco,PF,Rio de Janeiro,São Paulo (SP),Pessoa Física,False,False,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,Garantia de fábrica,NaN,NaN,11.249163
1,299896161723793383246777788797566040330,18.0,FIAT,STRADA,1.4 MPI WORKING CS 8V FLEX 2P MANUAL,2021,2021.0,53045.0,Manual,2,Picape,N,Branco,PJ,Palmas,Amazonas (AM),Loja,False,False,False,Aceita troca,Único dono,NaN,IPVA pago,Licenciado,NaN,NaN,NaN,11.014684
2,316180649972302128246133616457018378621,8.0,AUDI,Q5,2.0 TFSI GASOLINA BLACK S TRONIC,2018,2019.0,32486.0,Automática,4,Utilitário esportivo,N,Branco,PF,Goiânia,Goiás (GO),Pessoa Física,False,False,False,Aceita troca,NaN,NaN,IPVA pago,NaN,NaN,NaN,NaN,12.383594
3,222527157104148385909188217274642813298,16.0,CHEVROLET,CRUZE,1.4 TURBO LT 16V FLEX 4P AUTOMÁTICO,2019,2020.0,24937.0,Automática,4,Sedã,N,Cinza,PJ,Presidente Prudente,São Paulo (SP),Loja,False,False,False,Aceita troca,Único dono,NaN,IPVA pago,Licenciado,NaN,NaN,NaN,11.509156
4,160460343059850745858546502614838368036,8.0,FORD,ECOSPORT,1.5 TI-VCT FLEX SE AUTOMÁTICO,2019,2019.0,62074.0,Automática,4,Sedã,N,Branco,PJ,Limeira,São Paulo (SP),Loja,False,False,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,NaN,NaN,NaN,11.369226


In [29]:
cars_test.to_csv('predicted.csv', index=False)